In [1]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/ALLRoblox-onlyParentQuery.csv?token=GHSAT0AAAAAAB6VRXEYS2NLL5OEAF4HORA4ZALO6YQ')

In [3]:
df.head()

,type,url,title,author,date,timestamp,score,text,comment_id,parent_id
0,Post,https://www.reddit.com/r/roblox/comments/11iac...,"Parents, watch your damn kids.",Kratomamous,2023-03-04,1.677957e+09,70,"Parents, watch your damn kids.\nMy 7 y/o daugh...",NaN,NaN
1,comment,https://www.reddit.com/r/roblox/comments/11iac...,NaN,sneakysquid102,2023-03-05,1.677986e+09,83,Your kid is on the internet... There's no more...,jayvdhx,NaN
2,comment,https://www.reddit.com/r/roblox/comments/11iac...,NaN,ArcticLyrics,2023-03-04,1.677960e+09,28,The majority of life roleplay games have peopl...,jaxax5t,NaN
3,comment,https://www.reddit.com/r/roblox/comments/11iac...,NaN,TeamBoeing,2023-03-04,1.677971e+09,33,I would recommend games that are more focused ...,jaxzakc,NaN
4,comment,https://www.reddit.com/r/roblox/comments/11iac...,NaN,Legendaryy7,2023-03-04,1.677959e+09,14,usually life roleplay games like brookhaven ge...,jax7bws,NaN


In [4]:
df['text']

0       Parents, watch your damn kids.\nMy 7 y/o daugh...
1       Your kid is on the internet... There's no more...
2       The majority of life roleplay games have peopl...
3       I would recommend games that are more focused ...
4       usually life roleplay games like brookhaven ge...
                              ...                        
4055    I feel like they're a minority though.\n\nLike...
4056    To be fair we got little sex-craved goblins ru...
4057    \#\#\#\#\#\#\#\#\#\#\n\n\# Yes they do \#\n\n\...
4058                                 A vocal one, though.
4059    They don't if you put it in a line.  EDIT: Use...
Name: text, Length: 4060, dtype: object

In [5]:
#df['comment'].str.count('game').sum()

In [6]:
text = " ".join(df['text'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [7]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [8]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: ca nt with a frequency of 119


In [9]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
                  term   rank
6959            ca nt  124.0
53583        year old   98.0
34847     play roblox   71.0
35348  playing roblox   61.0
31169         nt know   60.0
17941      games like   55.0
34723      play games   52.0


In [10]:
df_ques = df['text'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['text'])

0       Parents, watch your damn kids.\nMy 7 y/o daugh...
32      There are parental controls you can use, as we...
51                 why are all the parts named "zarm"????
57           How in the hell is there a Funky Friday one?
69                                             toys r us?
                              ...                        
3986    you do realize roblox is operating at a loss a...
3995    oh damn, I thought creators were getting payed...
4012    An idea on Parental Controls\nThey should add ...
4016          Probably through the home router settings??
4020    I'm not a scripter, but I have a theory.\n\nHa...
Name: text, Length: 243, dtype: object


In [11]:
df['text_processed'] = df['text'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()


0    parents watch your damn kids\nmy 7 y/o daughte...
1    your kid is on the internet there's no more to...
2    the majority of life roleplay games have peopl...
3    i would recommend games that are more focused ...
4    usually life roleplay games like brookhaven ge...
Name: text_processed, dtype: object

In [12]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.text.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['parents', 'watch', 'damn', 'kids', 'daughter', 'thought', 'angel', 'compared', 'older', 'sister', 'found', 'got', 'banned', 'day', 'saying', 'bye', 'btc', 'someone', 'btc', 'apparently', 'abbreviation', 'bitch', 'way', 'get', 'around', 'word', 'filter', 'would', 'assume', 'didnt']


In [13]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
